In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-no

In [3]:
import nltk
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [4]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
no_finetuned = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
finetuned = T5ForConditionalGeneration.from_pretrained("gdrive/My Drive/T5_10k2", local_files_only=True)

In [6]:
inputs = tokenizer("How to make Classic Lasagna?", return_tensors="pt")
outputs = no_finetuned.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Mix 1 1/2 cups shredded mozzarella cheese, 1 1/2 cups shredded Parmesan cheese,']


In [7]:
question = "How to make Classic Lasagna?"
context = "Classic Lasagna is made by layering lasagna noodles with sauce"

# Format input text as expected by T5
input_text = f"How to make Classic Lasagna?"

input_ids = tokenizer.encode(input_text, return_tensors="pt")
# Generate the output using the model
outputs = finetuned.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)

# Decode the generated IDs back into text
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answer)

You can make Classic Lasagna with 1 lb. lasagna noodles, 1 lb. shredded mozzarella cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese


In [8]:
outputs = finetuned.generate(**inputs, max_length=512)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['You can make Classic Lasagna with 1 lb. lasagna noodles, 1 lb. shredded cheese, 1 lb. shredded mozzarella cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parmesan cheese, 1 lb. shredded Parme

In [22]:
inputs = tokenizer("Question: How to make kung-pao chicken?", return_tensors="pt")
outputs = no_finetuned.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Mix 1 tablespoon of kung-pao chicken broth with 1 tablespoon of water.']


In [24]:
outputs = finetuned.generate(**inputs, max_length=128)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['You can make Kung Pao Chicken with 1 lb. chicken, 1 lb. kung pao chicken, 1 lb. kung pao noodles, 1 lb. kung pao chicken, 1 lb. kung pao chicken, 1 lb. kung pao chicken, 1 lb. kung pao chicken, 1 lb. kung pao chicken, 1 lb. kung pao chicken, 1 lb. kung pao chicken, 1 l']


In [25]:
inputs = tokenizer("Question: I have some salmon and scallop. What can I cook for my family? Give me some ideas and instruction.", return_tensors="pt")
outputs = no_finetuned.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['I have salmon and scallops. I can cook them for my family. I can also make']


In [26]:
outputs = finetuned.generate(**inputs, max_length=128)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['You can make Salmon and Scallop with 1 c. salmon, 1 c. scallop, 1 c. steamed broccoli, 1 c. steamed kale, 1 c. steamed kale, 1 c. steamed kale, 1 c. steamed kale, 1 c. steamed kale, 1 c. steamed kale, 1 c. steamed kale, 1 c. steamed kale, 1 c. steamed kale']


In [13]:
inputs = tokenizer("Question: What can I make with penne pasta, red pepper, onion?", return_tensors="pt")
outputs = no_finetuned.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['fettuccine']


In [14]:
outputs = finetuned.generate(**inputs, max_length=128)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

["You can make Penne pasta with 1 lb. penne pasta, 1 red pepper, 1 onion, 1 tsp. salt, 1 tsp. pepper, 1 tsp. pepper, 1 tsp. pepper, 1 tsp. pepper. Here's the instruction: Preheat oven to 350°. In a large bowl, combine pasta, onion, pepper, pepper, pepper, and pepper. Mix well. Add pasta and cook for about 5 minutes. Add salt and pepper. Cook for another 5 minutes. Add pasta and cook for another 5"]


In [15]:
def inference(sentence, model):
  task_prefix = "answer the question: "
  inputs = tokenizer([task_prefix + sentence], return_tensors="pt",  max_length=512)
  output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,
  )
  return tokenizer.batch_decode(output_sequences, skip_special_tokens=True, max_length=512)

In [16]:
inference("I have chicken breast, cabbage, and potato. Give me some cooking ideas . ", finetuned)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['You can make Chicken and Potato Salad with 1 c. chicken breast, 1 c']